# drawing-emus-v2
neatened version of notebook `drawing-emus.ipynb`

In [ ]:
# stock imports
import pickle
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import scipy

##plotting
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import corner

from scripts import prior_funcs, utils

# plt.style.use('dark_background')
plt.style.use("Solarize_Light2")
plt.rcParams.update({"axes.edgecolor": "black"})
plt.rcParams.update({"text.color": "black"})
plt.rcParams.update({"axes.labelcolor": "black"})
plt.rcParams.update({"xtick.color": "black"})
plt.rcParams.update({"ytick.color": "black"})
plt.rcParams.update({"font.family": "monospace"})

#script imports
from scripts import prior_funcs, utils

from scripts.pitchfuncs_ultra_pca_v2 import emulator
from scripts.pitchfuncs_ultra_pca_v2 import ultra_ns_vector_surface
from scripts.pitchfuncs_ultra_pca_v2 import ultra_ns_popslice
from scripts.pitchfuncs_ultra_pca_v2 import ultra_ns_popwalk


import logging
logging.getLogger('ultranest').setLevel(logging.WARNING)

os.environ["CUDA_VISIBLE_DEVICES"]="1"

physical_devices = tf.config.list_physical_devices("GPU") 

#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


tf.config.experimental.set_memory_growth(physical_devices[0], True)

gpu0usage = tf.config.experimental.get_memory_info("GPU:0")["current"]

In [ ]:
pitchfork_name = "nu6-40_elu_nonorm_feh"
pitchfork = emulator(pitchfork_name)

with open("pitchfork/" +pitchfork_name+ ".pkl", 'rb') as fp:
     pitchfork_info = pickle.load(fp)

pitchfork_ranges = pitchfork_info['parameter_ranges']

## define priors
start off by defining some priors that we'll use to inform sampling of emu parameters. priors should:
- be bounded to the edges of the model grid to avoid neural network extrapolation (beta distributions are our friends here)
- be (roughly) the same as ones we'll use later in sampling

In [ ]:
labels = ["initial_mass", "initial_Zinit", "initial_Yinit", "initial_MLT", "star_age", "a", "b"]

mass_prior = prior_funcs.custom_beta_prior(0.8, 1.2, a=2, b=5)

Zinit_prior = prior_funcs.custom_beta_prior(0.004, 0.038, a=2, b=5)

Yinit_prior = prior_funcs.custom_beta_prior(0.24, 0.32, a=2, b=5)

MLT_prior = prior_funcs.custom_beta_prior(1.7, 2.5, a=1.2, b=1.2)

age_prior = prior_funcs.custom_beta_prior(0.03, 14, a=1.2, b=1.2)

a_prior = prior_funcs.custom_uniform_prior(-10, 2)

b_prior = prior_funcs.custom_uniform_prior(4.4, 5.25)


priors = [mass_prior, Zinit_prior, Yinit_prior, MLT_prior, age_prior, a_prior, b_prior]

mass_prior = prior_funcs.custom_uniform_prior(0.86, 1)

Zinit_prior = prior_funcs.custom_uniform_prior(0.008, 0.016)

Yinit_prior = prior_funcs.custom_uniform_prior(0.25, 0.27)

MLT_prior = prior_funcs.custom_uniform_prior(1.9, 2.3)

age_prior = prior_funcs.custom_uniform_prior(2, 12)

a_prior = prior_funcs.custom_uniform_prior(-0.1, 0.1)

b_prior = prior_funcs.custom_uniform_prior(4.4, 5.25)

hare_draws = [mass_prior, Zinit_prior, Yinit_prior, MLT_prior, age_prior, a_prior, b_prior]

size = 10000
plot_prior_samples = np.array([prior.rvs(size=size) for prior in priors])
legend_labels = []
corner_color = "black"
figure = corner.corner(
    plot_prior_samples.T,
    labels=labels,
    color=corner_color,
    hist_kwargs={"density": True},
    smooth=True,
)

plot_hare_draw_samples = np.array([hare_draw.rvs(size=size) for hare_draw in hare_draws])
corner.corner(
    plot_hare_draw_samples.T,
    labels=labels,
    color="red",
    fig=figure,
    hist_kwargs={"density": True},
    smooth=True,
)

plt.suptitle("prior samples")
plt.show()

## import grid
now we'll import the grid of stellar models to use when we draw emus

In [ ]:
df_full = pd.concat(
    [
        pd.read_hdf(
            "/home/oxs235/datastorage/repos_data/ojscutt/mesa_ajl/data/grid2p5a/grid.h5",
            key=f"m{0.8 + i*0.02:.2f}",
        )
        for i in range(21)
    ],
    ignore_index=True,
)
df = df_full[df_full["log_g"] >= 3.6]
df = df[df["star_age"] <= 14]
df = df[df["frac_age"] >= 0.015]

inputs = ["initial_mass", "initial_Zinit", "initial_Yinit", "initial_MLT", "star_age"]
outputs = (
    ["radius", "luminosity", "star_feh"]
    + [f"nu_0_{i}" for i in range(6, 41)]
    + ["nu_max"]
)

df = df.loc[(df[[f"nu_0_{i}" for i in range(6, 41)]] != 0).all(axis=1)]

cols = inputs + outputs

df = df[cols]

df.dropna(inplace=True)

df["calc_effective_T"] = utils.calc_Teff(df["luminosity"], df["radius"])

In [ ]:
df.describe()

## draw emus
here's where we use our priors to inform nearest-neighbour sampling of emus from the imported grid

In [ ]:
emus_to_sample = 10


prior_samples = np.array([hare_draw.rvs(size=emus_to_sample) for hare_draw in hare_draws])


def corner_legend(color, label):
    return [
        Line2D(
            [0],
            [0],
            marker="",
            color=color,
            markerfacecolor=color,
            markersize=10,
            label=label,
        )
    ]


# plotting samples over priors:
legend_labels = []

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

priors_color = "black"
figure = corner.corner(
    plot_prior_samples.T,
    labels=labels,
    color=priors_color,
    hist_kwargs={"density": True},
    smooth=True,
)
legend_labels += corner_legend(priors_color, "prior")

emus_color = colors[0]
corner.corner(
    prior_samples.T,
    labels=labels,
    color=emus_color,
    hist_kwargs={"density": True},
    fig=figure,
    smooth=True,
)
legend_labels += corner_legend(emus_color, "emus")

plt.legend(handles=legend_labels, bbox_to_anchor=(0, 1, 0.48, 0.31))
plt.suptitle("prior with drawn emus")
plt.show()

In [ ]:
from scipy.spatial.distance import cdist
import os


def closest_grid_point(
    point,
    grid_df,
    a,
    b,
    columns=[
        "initial_mass",
        "initial_Zinit",
        "initial_Yinit",
        "initial_MLT",
        "star_age",
    ],
):

    closest_point = grid_df.iloc[[cdist([point], grid_df[columns].values).argmin()]].copy()

    closest_point['a'] = a
    closest_point['b'] = b
    
    return closest_point


def nu_max_n(emu_sample):
    nu_max = emu_sample["nu_max"]
    nu_max_header = (
        np.abs(emu_sample[[f"nu_0_{i}" for i in range(6, 41)]] - nu_max.values[0])
        .min()
        .idxmin()
    )
    emu_sample_new = emu_sample.copy()
    emu_sample_new.loc[:, "nu_max_n"] = nu_max_header.replace("nu_0_", "")
    return emu_sample_new


for emu_idx in range(emus_to_sample):
    emu_sample = closest_grid_point(prior_samples[:5, emu_idx], df, 0, 4.8)
    emu_sample = nu_max_n(emu_sample)
    emu_sample[['calc_effective_T', 'luminosity', 'star_feh'] + [f'nu_0_{i}' for i in range(6,41)]] = pitchfork.predict(emu_sample[['initial_mass', 'initial_Zinit', 'initial_Yinit', 'initial_MLT', 'star_age']].values)
    path = f"nest/emu{emu_idx}"
    if not os.path.exists(path):
        os.mkdir(path)
        print(f"{path} created!")
    else:
        print(f"{path} already exists", end="\r")
    emu_sample.to_json(path + f"/emu{emu_idx}.json")